고차원 공간의 특징
- 3차원을 초과하는 고차원 공간의 구조를 상상하기는 매우 어렵다.
- 고차원 공간의 경우 두 지점 사이의 거리가 매우 멀어질 수 있으며, 이는 특성 수가 많은 데이터들 사이의 거리가 매우 멀어질 수 있음을 의미한다.

차원의 저주
- 머신러닝에서 다루는 많은 문제가 수 천에서 수 백만의 특성을 가진 데이터를 다룬다.
    - 이는 데이터셋의 특성이 이루는 공간이 수 천에서 수 백만 차원의 공간임을 의미한다.
- 수 천에서 수 백만의 차원을 갖는 데이터셋을 훈련시키는 데에는 엄청난 시간과 비용이 발생한다.
- 또한, 데이터 사이가 멀기에 새로운 데이터에 대한 예측값을 계산하기 위해 많은 추정 과정을 거처야 한다.
    - 따라서 과대적합 위험도가 커지는 등 훈련이 매우 어려워진다.
- 데이터 사이의 거리를 충분히 작게 만들기 위해 천문학적 크기의 데이터셋이 요구되기에 사실상 해결이 불가능하다.

차원 축소
- 고차원의 데이터셋을 저차원의 데이터셋으로 변환하는 과정이다.
- 많은 실전 문제의 데이터셋의 특성들이 이루는 차원을 혁신적으로 줄일 수 있다.
- 예
    - MNIST 데이터셋의 경우 손글씨 이미지의 테두리 영역은 거의 항상 흰색이기에, 숫자 인식을 위해 아무런 역할을 하지 않는다.
    - 따라서 그 영역을 제외한 특성들의 차원만을 이용할 수 있다.
        - 실제로 784(18*18)개의 픽셀 특성 대신 154개의 픽셀 특성만을 사용해도 숫자 분류를 진행할 수 있음을 아래에서 확인할 수 있다.
    
장점과 단점
- 차원축소를 진행하면, 정보를 일부 또는 상당히 잃게 된다.
    - 따라서 적절한 수의 차원으로 축소를 진행해야 하며, 어느 정도로 줄이는 것이 적절한지 알아내야 한다.
- 차원축소를 진행하게 되면 훈련 속도가 빨라진다.
- 2, 3차원으로 축소할 경우 데이터 시각화가 가능해져서 데이터들의 패턴 인식이 가능하다.

사영 기법
- 특이값 분해를 이용한 사영 기법을 설명하기 위해 3차우너 공간에 아래와 같이 분포된 60개의 점을 2차우너으로 사영하는 예제를 이용한다.
- 언급된 60개의 3차원 데이터는 아래와 같이 (60, 3) 모양의 행렬 X로 생성한다.
    - 생성되는 데이터의 x, y좌표는 사인, 코사인 함수를 조합하며, 수학적으로 큰 의미는 없다.

In [ ]:
import numpy as np
np.random.seed(4)

m=60
noise=0.1
X=np.empty((m, 3))

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise + np.random.randn(m) / 2 # x 좌표
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2 # y 좌표

w1, w2 = 0.1, 0.3
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m) # z 좌표 (초평면 + 잡음)

PCA(주성분 분석)
- 데이터셋의 주성분은 데이터 샘플들의 분산을 최대로 유지하는 축을 가리키며, 차원의 개수만큼 주성분이 존재한다.
- 분산을 최대로 유지하는 축이 첫번째 주성분이며, 이후 축은 이전 축에 수직이면서, 동시에 남은 분산을 최대한 보존하는 축으로 지정한다.
- 이 과정을 차원 수 만큼의 주성분을 찾을 때까지 반복한다.

차원 축소
- 차원축소는 PCA를 통해 찾은 주성분의 일부로 구성된 초평면으로 데이터 샘플을 사용하는 과정을 의미한다.
- 첫번째부터 d번째까지의 주성분을 축으로 사용해서 생성되는 d차원의 공간으로 데이터셋이 사영된다.
    - 따라서 3차원(3D) 데이터를 2차원(2D) 공간으로 보내려면 예를 들어 아래 그림의 회색 평면을 구성하는 두 개의 축에 해당하는 주성분을 찾아야 한다.

특이값 분해(SVD)
- 데이터셋의 분산에 대한 주성분은 데이터셋 행렬의 특이값 분해(SVD)를 이용하여 쉽게 구할 수 있다.
- 사이킷런의 PCA 모델이 특이값 분해를 이용하여 주성분을 바로 계산할 수 있지만, 아래에선 먼저 특이값 분해를 사용하여 직접 주성분을 구하는 과정을 살펴본다.
    - 평균값(mean)이 0이라는 가정하에 특이값 분해를 진행해야 하기에, 아래 코드를 먼저 실행한다.
```
X_centered = X - X.mean(axis=0)
```

In [ ]:
X_centered = X - X.mean(axis=0)

특이값 분해를 진행하는 numpy.linalg.svd() 함수를 이용하여 X_centered에 대하여 특이값 분해를 진행한다.

In [ ]:
U, s, Vt = np.linalg.svd(X_centered)

각 행렬의 모양을 확인하면 아래와 같다.

In [ ]:
print(f"U shape: {U.shape}")
print(f"s shape: {s.shape}")
print(f"Vt shape: {Vt.shape}")

특이값 분해가 제대로 이뤄진 것인지는 아래와 같이 검증할 수 있다.
- 행렬의 곱셈을 진행하려면, 먼저 s를 (60, 3)모양의 대각행렬로 만들어야 한다.

In [ ]:
S = np.zeros(X_centered.shape)
S[:3, :3] = np.diag(s)

컴퓨터를 이용한 부동소수점 연산은 일반적으로 100% 정확하지 않다.
- numpy.allclose() 함수는 지정된 오차범위 안에서 두 부동소수점의 일치여부를 판단한다.

In [ ]:
np.allclose(X_centered, U.dot(S).dot(Vt))

주성분은 앞서 특이값 분해로 얻은 행렬 V의 열 벡터에 해당한다.

2차원으로 사영하기

3차원 공간의 데이터를 2차원으로 보내려면 앞서 구한 2개의 주성분을 선택한 후, 두 축에 의해 생성되는 초평면으로 사영해야 한다.
- 2차원으로 사영하려면 행렬 V의 0번과 1번 두 개의 열을 선택한 후, X_centered를 두 개의 주성분으로 구성된 행렬과 곱한다.

In [ ]:
w2 = Vt.T[:, :2] # 주성분 2개 선택
X2D = X_centered.dot(w2) # 데이터셋을 2차원으로 사영한다.

사영된 데이터셋을 사이킷런의 PCA 모델을 이용하여 구한 값과 비교를 위해 저장한다.

In [ ]:
X2D_using_svd = X2D

사이킷런의 PCA 모델 활용
- 앞서, 특이값 분해를 직접 활용하여 2차원으로 사영하는 과정을 사이킷런의 PCA 모델을 이용하여 간단히 해결할 수 있다.
    - 평균값을 0으로 맞추는 과정도 PCA 모델이 알아서 처리한다.
- fit()
    - 평균값을 0으로 맞춘 후, 특이값 분해를 이용하여 주성분을 찾는다.
- transform()
    - 찾은 주성분 2개를 이용하여 2차원 공간으로 데이터를 사영하는 과정은 다음과 같다.
        - 1. 평균값을 0으로 맞춘다.
        - 2. 특이값 분해로 찾은 주성분을 이용하여 2차원으로 사영한다.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components= 2) # 2차원으로 사영하는 모델
X2D = pca.fit_transform(X)

사영된 데이터의 축별 평균은 거의 0이다.

In [ ]:
print(X2D.mean(0))

사이킷런을 이용하여 PCA를 진행할 때 주성분 축이 반대 방향으로 지정되는 경우가 발생할 수 있다.
- 아래의 코드는 그런 일이 벌어졌음을 알려준다.

In [ ]:
print(np.allclose(X2D, X2D_using_svd))

print(np.allclose(X2D, -X2D_using_svd))

실제로 훈련된 PCA가 알고 있는 주성분과 앞서 특이값 분해로 얻은 주성분이 서로 다른 방향을 가리킨다.

In [ ]:
print(pca.components_)

print(Vt[:2])

print(np.allclose(pca.components_, -Vt[:2]))

재구성 오차
- 아래 코드는 2차원으로 사영된 데이터를 다시 3차원으로 복원한다.

In [ ]:
X3D_inv = pca.inverse_transform(X2D)

2차원 공간으로 사영되면서, 정보손실이 있었기에 3차우너으로 복원한 값과 원래 3차우너 값 사이에 오차가 존재한다.

In [ ]:
np.allclose(X3D_inv, X)

재구성 오차는 사영 전과 사영 후 원래 공간으로 복원한 데이터 사이의 평균제곱오차(MSE)로 계산한다.

In [ ]:
np.mean(np.sum(np.square(X3D_inv - X), axis=1))

inverse_transform() 메서드가 내부에서는 사영할 때 사용된 행렬의 전치행렬을 사영된 데이터셋에 곱한다.

In [ ]:
X3D_inv_using_svd = X2D_using_svd.dot(Vt[:2, :])

앞서 구한 inverse_transform() 메서드의 결과와 비교하기 위해, 다시 데이터셋의 평균을 뺀다.

In [ ]:
X.mean(0)

PCA 모델의 mean_ 속성에도 동일한 값이 저장되어 있다.

In [ ]:
pca.mean_

두 값을 비교하면 아래와 같이 일치한다.

In [ ]:
np.allclose(X3D_inv_using_svd, X3D_inv - pca.mean_)

설명 분산 비율

설명 분산 비율은 각 주성분의 축에 대한 데이터셋의 분산 비율을 의미한다.
- PCA 객체의 explained_variance_ratio_ 속성에 사영에 사용된 주성분별 설명 분산 비율이 저장되어 있다.
- 첫째 주성분의 축에 대한 데이터셋의 분산 비율: 84.25%
- 둘째 주성분의 축에 대한 데이터셋의 분산 비율: 14.63%

In [ ]:
pca.explained_variance_ratio_

즉, 2차원으로 사영한 결과 0.9% 정도의 분산을 잃는다.

In [ ]:
1 - pca.explained_variance_ratio_.sum()

특이값 분해로 생성된 행렬 s를 이용하여, 모든 주성분에 대한 설명 분산 비율을 계산할 수 있다.

In [ ]:
np.square(s) / np.square(s).sum()

지금까지 사용된 그래프를 그리는 코드는 아래와 같다.

In [ ]:
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

3차원 공간에서의 2차원 초평면 좌표

In [ ]:
axes = [-1.8, 1.8, -1.3, 1.3, -1.0, 1.0]

x1s = np.linspace(axes[0], axes[1], 10)
x2s = np.linspace(axes[2], axes[3], 10)
x1, x2 = np.meshgrid(x1s, x2s) # x축 좌표

C = pca.components_
R = C.T.dot(C)
z = (R[0, 2] * x1 + R[1, 2] * x2) / (1 - R[2, 2]) # z축 좌표

그래프 그리기
- 3D 데이터셋, 평면, 그리고 사영 과정

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(6, 3.8))
ax = fig.add_subplot(111, projection='3d')

# 3차원 데이터 산점도 그리기: 초평면 윗쪽, 아랫쪽 데이터 구분
X3D_above = X[X[:, 2] > X3D_inv[:, 2]]    # 초평면 윗쪽 데이터
X3D_below = X[X[:, 2] <= X3D_inv[:, 2]]   # 초평면 아랫쪽 데이터

ax.plot(X3D_above[:, 0], X3D_above[:, 1], X3D_above[:, 2], "bo")
ax.plot(X3D_below[:, 0], X3D_below[:, 1], X3D_below[:, 2], "go", alpha=0.5)

# 주성분의 축 그리기
# np.linalg.norm(C, axis=0)
# ax.add_artist(Arrow3D([0, C[0, 0]],[0, C[0, 1]],[0, C[0, 2]], mutation_scale=15, lw=1, arrowstyle="-|>", color="k"))
# ax.add_artist(Arrow3D([0, C[1, 0]],[0, C[1, 1]],[0, C[1, 2]], mutation_scale=15, lw=1, arrowstyle="-|>", color="k"))
# ax.plot([0], [0], [0], "k.")

# 초평면 그리기
ax.plot_surface(x1, x2, z, alpha=0.2, color="k")

# 초평면으로 사영된 데이터셋 그리기
ax.plot(X3D_inv[:, 0], X3D_inv[:, 1], X3D_inv[:, 2], "k+")
ax.plot(X3D_inv[:, 0], X3D_inv[:, 1], X3D_inv[:, 2], "k.")

# 사영 과정
for i in range(m):
    if X[i, 2] > X3D_inv[i, 2]:
        ax.plot([X[i][0], X3D_inv[i][0]], [X[i][1], X3D_inv[i][1]], [X[i][2], X3D_inv[i][2]], "r-")
    else:
        ax.plot([X[i][0], X3D_inv[i][0]], [X[i][1], X3D_inv[i][1]], [X[i][2], X3D_inv[i][2]], "r-", color="#505050")

# 축 레이블 표기
ax.set_xlabel("$x_1$", fontsize=18, labelpad=10)
ax.set_ylabel("$x_2$", fontsize=18, labelpad=10)
ax.set_zlabel("$x_3$", fontsize=18, labelpad=10)
ax.set_xlim(axes[0:2])
ax.set_ylim(axes[2:4])
ax.set_zlim(axes[4:6])


plt.show()

2차원 공간으로 사영된 데이터셋 그리기

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal')

ax.plot(X2D[:, 0], X2D[:, 1], "k+")
ax.plot(X2D[:, 0], X2D[:, 1], "k.")
ax.plot([0], [0], "ko")
ax.arrow(0, 0, 0, 1, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
ax.arrow(0, 0, 1, 0, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
ax.set_xlabel("$z_1$", fontsize=18)
ax.set_ylabel("$z_2$", fontsize=18, rotation=0)
ax.axis([-1.5, 1.3, -1.2, 1.2])
ax.grid(True)

사영기법의 한계
- 사영기법이 차원축소의 최선이 아닌 경우가 존재한다.
- 이어서 설명하는 롤케이크의 경우처럼 데이터셋을 구성하는 저차원 공간이 꼬이거나 뒤집혀 있는 경우, 사영기법을 적용하면 오히려 문제를 어렵게 만들 수 있다.

롤케이크 그리기
- 아래는 롤케이크(스위스 롤) 데이터셋을 생성한다.
- X
    - 롤케이크 데이터셋(3차원)
- t
    - 롤케이크 생성에 사용되는 1차원 데이터셋

In [ ]:
from sklearn.datasets import make_swiss_roll
X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)

In [ ]:
axes = [-11.5, 14, -2, 23, -12, 15]

fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(111, projection="3d")

# 롤케이크 데이터셋 산점도
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=t, cmap=plt.cm.hot)   # t에 의존한 색상 지정

ax.view_init(10, -70)
ax.set_xlabel("$x_1$", fontsize=18)
ax.set_ylabel("$x_2$", fontsize=18)
ax.set_zlabel("$x_3$", fontsize=18)
ax.set_xlim(axes[0:2])
ax.set_ylim(axes[2:4])
ax.set_zlim(axes[4:6])

plt.show()

롤케이크 사영하기와 펼치기

롤케이크 모양의 데이터셋을 2차원으로 사영해도 분류를 깨끗하게 실행할 수 없다.
- 사영: x1과 x2를 주성분 축으로 사용한다. 즉, x3을 무시한다.

In [ ]:
fig = plt.figure(figsize=(5.5, 4))

ax = fig.add_subplot(111)
ax.scatter(X[:, 0], X[:, 1], c=t, cmap=plt.cm.hot)
ax.axis(axes[:4])
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$x_2$", fontsize=18, rotation=0)
plt.grid(True)

plt.show()

하지만, 아래 그림처럼 돌돌 말린 데이터셋을 평평하게 펼치면, 보다 쉽게 분류가 가능하다.

In [ ]:
fig = plt.figure(figsize=(5.5, 4))

ax = fig.add_subplot(111)
ax.scatter(t, X[:, 1], c=t, cmap=plt.cm.hot)
ax.axis([4, 15, axes[2], axes[3]])
plt.ylabel("$x2$", fontsize=18)
plt.xlabel("$z_1$", fontsize=18)
plt.grid(True)

plt.show()

다양체 학습
- 롤케이크는 3차우너 공간에 존재한다.
- 반면, 위 그림에서 보듯 롤케이크를 구성하는 데이터셋 자체는 2차우너 평면을 구부린 형태를 갖는다.
- 이처럼 저차우너 공간이 구부리고 꼬인 형태로 보다 고차원의 공간에 위치하면, 그런 저차원 공간을 다양체(manifold)라고 한다.
- 즉, 롤케이크는 2차원 다양체가 된다.

다양체 학습(manifold learning)
- 고차원 공간에 존재하는 데이터셋의 다양체 성질을 학습하는 것을 의미한다.

다양체 가정
- 고차원의 데이터셋이 대부분 저차원의 다양체와 비슷한 형태를 갖는다 라는 가정을 다양체 가정 또는 다양체 가설이라고 한다.
- 다양체 가정과 더불어 저차원의 다양체로 표현할 경우 문제가 보다 쉬워질 것이다 라는 가정도 함께 사용된다.
- 앞서 살펴본 롤케이크의 경우가 이 가정을 만족시킨다.
    - 하지만, 이 가정또한 항상 성립하지는 않음을 아래 그래프가 잘 보여준다.
        - 위쪽 그림
            - 3차원 롤케이크 모양의 데이터셋
        - 아래쪽 그림
            - 돌돌 말린 모양을 펼친 형태의 2차원 데이터셋

In [ ]:
from matplotlib import gridspec

axes = [-11.5, 14, -2, 23, -12, 15]

x2s = np.linspace(axes[2], axes[3], 10)
x3s = np.linspace(axes[4], axes[5], 10)
x2, x3 = np.meshgrid(x2s, x3s)

fig = plt.figure(figsize=(6, 5))
ax = plt.subplot(111, projection='3d')

positive_class = X[:, 0] > 5
X_pos = X[positive_class]
X_neg = X[~positive_class]
ax.view_init(10, -70)
ax.plot(X_neg[:, 0], X_neg[:, 1], X_neg[:, 2], "y^")
ax.plot_wireframe(5, x2, x3, alpha=0.5)
ax.plot(X_pos[:, 0], X_pos[:, 1], X_pos[:, 2], "gs")
ax.set_xlabel("$x_1$", fontsize=18)
ax.set_ylabel("$x_2$", fontsize=18)
ax.set_zlabel("$x_3$", fontsize=18)
ax.set_xlim(axes[0:2])
ax.set_ylim(axes[2:4])
ax.set_zlim(axes[4:6])
plt.show()

fig = plt.figure(figsize=(5, 4))
ax = plt.subplot(111)

plt.plot(t[positive_class], X[positive_class, 1], "gs")
plt.plot(t[~positive_class], X[~positive_class, 1], "y^")
plt.axis([4, 15, axes[2], axes[3]])
plt.xlabel("$z_1$", fontsize=18)
plt.ylabel("$z_2$", fontsize=18, rotation=0)
plt.grid(True)
plt.show()

반면, 아래의 경우 저차원 다양체로 말린 부분을 펼치면 분류 문제가 매우 단순해지는 것을 알 수 있다.

In [ ]:
from matplotlib import gridspec

axes = [-11.5, 14, -2, 23, -12, 15]

x2s = np.linspace(axes[2], axes[3], 10)
x3s = np.linspace(axes[4], axes[5], 10)
x2, x3 = np.meshgrid(x2s, x3s)

fig = plt.figure(figsize=(6, 5))
ax = plt.subplot(111, projection='3d')

positive_class = 2 * (t[:] - 4) > X[:, 1]
X_pos = X[positive_class]
X_neg = X[~positive_class]
ax.view_init(10, -70)
ax.plot(X_neg[:, 0], X_neg[:, 1], X_neg[:, 2], "y^")
ax.plot(X_pos[:, 0], X_pos[:, 1], X_pos[:, 2], "gs")
ax.set_xlabel("$x_1$", fontsize=18)
ax.set_ylabel("$x_2$", fontsize=18)
ax.set_zlabel("$x_3$", fontsize=18)
ax.set_xlim(axes[0:2])
ax.set_ylim(axes[2:4])
ax.set_zlim(axes[4:6])
plt.show()

fig = plt.figure(figsize=(5, 4))
ax = plt.subplot(111)

plt.plot(t[positive_class], X[positive_class, 1], "gs")
plt.plot(t[~positive_class], X[~positive_class, 1], "y^")
plt.plot([4, 15], [0, 22], "b-", linewidth=2)
plt.axis([4, 15, axes[2], axes[3]])
plt.xlabel("$z_1$", fontsize=18)
plt.ylabel("$z_2$", fontsize=18, rotation=0)
plt.grid(True)
plt.show()

PCA

주성분 분석은 가장 많이 사용되는 차원축소 알고리즘이다.
주성분 분석의 핵심은 훈련 데이터셋의 분산을 최대한 보존하는 방향으로 차원축소를 진행하는 것이다.

분산 보존

아래 그림은 분산을 최대로 보존하는 주성분의 의미를 잘 보여준다.
- c1
    - 첫째 주성분, 이 축으로 사영할 경우 분산이 가장 많이 보존된다.
- c2   
    - 둘째 주성분, c1과 수직을 이루면서 분산을 가장 많이 보존하는 축

2차원 데이터셋이 두 개의 주성분만 존재하며, 첫 번째 주성분이 정해지면 두 번째 주성분은 자동으로 정해진다.

In [ ]:
angle = np.pi / 5
stretch = 5
m = 200

np.random.seed(3)
X = np.random.randn(m, 2) / 10
X = X.dot(np.array([[stretch, 0],[0, 1]])) # stretch
X = X.dot([[np.cos(angle), np.sin(angle)], [-np.sin(angle), np.cos(angle)]]) # rotate

u1 = np.array([np.cos(angle), np.sin(angle)])
u2 = np.array([np.cos(angle - 2 * np.pi/6), np.sin(angle - 2 * np.pi/6)])
u3 = np.array([np.cos(angle - np.pi/2), np.sin(angle - np.pi/2)])

X_proj1 = X.dot(u1.reshape(-1, 1))
X_proj2 = X.dot(u2.reshape(-1, 1))
X_proj3 = X.dot(u3.reshape(-1, 1))

# 왼편 그래프
plt.figure(figsize=(8,4))
plt.subplot2grid((3,2), (0, 0), rowspan=3)
plt.plot([-1.4, 1.4], [-1.4*u1[1]/u1[0], 1.4*u1[1]/u1[0]], "k-", linewidth=1)
plt.plot([-1.4, 1.4], [-1.4*u2[1]/u2[0], 1.4*u2[1]/u2[0]], "k--", linewidth=1)
plt.plot([-1.4, 1.4], [-1.4*u3[1]/u3[0], 1.4*u3[1]/u3[0]], "k:", linewidth=2)
plt.plot(X[:, 0], X[:, 1], "bo", alpha=0.5)
plt.axis([-1.4, 1.4, -1.4, 1.4])
plt.arrow(0, 0, u1[0], u1[1], head_width=0.1, linewidth=5, length_includes_head=True, head_length=0.1, fc='k', ec='k')
plt.arrow(0, 0, u3[0], u3[1], head_width=0.1, linewidth=5, length_includes_head=True, head_length=0.1, fc='k', ec='k')
plt.text(u1[0] + 0.1, u1[1] - 0.05, r"$\mathbf{c_1}$", fontsize=22)
plt.text(u3[0] + 0.1, u3[1], r"$\mathbf{c_2}$", fontsize=22)
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$x_2$", fontsize=18, rotation=0)
plt.grid(True)

# 오른편 상단 그래프: C1 축으로 사영한 데이터 그래프
plt.subplot2grid((3,2), (0, 1))
plt.plot([-2, 2], [0, 0], "k-", linewidth=1)
plt.plot(X_proj1[:, 0], np.zeros(m), "bo", alpha=0.3)
plt.gca().get_yaxis().set_ticks([])
plt.gca().get_xaxis().set_ticklabels([])
plt.axis([-2, 2, -1, 1])
plt.grid(True)

# 오른편 중간 그래프: C1과 C2 사이에 있는 파선으로 된 축으로 사영한 데이터 그래프
plt.subplot2grid((3,2), (1, 1))
plt.plot([-2, 2], [0, 0], "k--", linewidth=1)
plt.plot(X_proj2[:, 0], np.zeros(m), "bo", alpha=0.3)
plt.gca().get_yaxis().set_ticks([])
plt.gca().get_xaxis().set_ticklabels([])
plt.axis([-2, 2, -1, 1])
plt.grid(True)

# 오른편 하단 그래프: C2 축으로 사영한 데이터 그래프
plt.subplot2grid((3,2), (2, 1))
plt.plot([-2, 2], [0, 0], "k:", linewidth=2)
plt.plot(X_proj3[:, 0], np.zeros(m), "bo", alpha=0.3)
plt.gca().get_yaxis().set_ticks([])
plt.axis([-2, 2, -1, 1])
plt.xlabel("$z_1$", fontsize=18)
plt.grid(True)

plt.show()

PCA 활용: MNIST 데이터 압축


fetch_openml() 함수는 지정된 데이터셋과 관련된 다양한 정보를 담은 사전 객체(Bunch 객체)를 반환하며, 특성과 타깃 데이터셋은 각각 다음 키의 값으로 지정되어 있다.
- data
    - 특성 데이터셋
- target
    - 타깃 데이터셋

특성 데이터셋과 타깃 데이터셋이 판다스의 DataFrame 또는 Series 객체로 저장된다.
- 여기서는 각 데이터셋을 넘파이 어레이로 얻기 위해 as_frame=False 옵션을 지정한다.

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml("mnist_784", version=1, as_frame=False)

print(type(mnist))

아래 코드는 특성 데이터셋과 타깃 데이터셋 넘파이 어레이로 지정한다.

In [ ]:
mnist.target = mnist.target.astype(np.uint8)

X = mnist["data"]
y = mnist["target"]

훈련 세트와 테스트 세트로 구분한다.
- 훈련 세트
    - 전체 데이터셋의 75%, 52500개의 샘플
- 테스트 세트
    - 나머지 데이터

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

설명 분산 비율과 차원 축소

차원 축소를 위한 적절한 차원을 찾기 위해 설명 분산 비율이 95%가 되는 지점까지 몇 개의 주성분이 필요한가를 계산한다.
- MNIST 데이터셋의 경우 설명 분산 비율이 95%가 되도록 하려면 154개의 주성분이 필요함이 아래와 같이 확인된다.

In [ ]:
pca = PCA()
pca.fit(X_train)

cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
print(d)

MNIST 데이터셋과 관련된 설명 분산 비율과 차원과의 관계를 그래프로 그리면 다음과 같다.
- (154, 95) 좌표의 지점 표시
- 엘보우
    - 설명 분산 비율 그래프의 증가도가 급격하게 떨어지는 지점

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(cumsum, linewidth=3)                                # 설명 분산 비율 그래프(파랑색 곡선)

plt.axis([0, 400, 0, 1])
plt.xlabel("Dimensions")
plt.ylabel("Explained Variance")

plt.plot(d, 0.95, "ko")                                     # (154, 95) 좌표의 점
plt.plot([d, d], [0, 0.95], "k:")                           # 수직 점선
plt.plot([0, d], [0.95, 0.95], "k:")                        # 수평 점선
plt.annotate("Elbow", xy=(65, 0.85), xytext=(70, 0.7),      # Elbow 주석
             arrowprops=dict(arrowstyle="->"), fontsize=16)

plt.grid(True)

plt.show()

차원을 직접 지정하는 대신 n_components 옵션에 설명 분산 비율을 지정하면 자동으로 해당 분산 비율을 만족하는 최소 차원으로 데이터셋을 사영할 수 있다.

In [ ]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

몇 개의 주성분이 사용되었는가는 n_components_ 속성이 알고 있다.

In [ ]:
pca.n_components_

사영에 사용된 설명 분산 비율의 합은 아래와 같이 95%를 넘는다.

In [ ]:
np.sum(pca.explained_variance_ratio_)

재구성 오차
- 재구성 오차는 170,000 정도로 계산된다.

In [ ]:
X_recovered = pca.inverse_transform(X_reduced)

np.mean(np.sum(np.square(X_recovered - X_train), axis=1))

앞서 확인한 차원 154를 이용하여 차원축소를 진행하더라도 재구성 오차가 거의 비슷하다.

In [ ]:
pca = PCA(n_components=154)
X_reduced = pca.fit_transform(X_train)

X_recovered = pca.inverse_transform(X_reduced)
np.mean(np.sum(np.square(X_recovered - X_train), axis=1))

실제로 원본 손글씨 숫자와 재구성된 숫자 이미지를 비교해보면, 화질이 조금 떨어지긴 했지만 숫자 인식에는 전혀 문제가 없다는 것을 확인할 수 있다.
- plot_digits() 함수는 여러 개의 손글씨 데이터를 하나로 묶어서 보여준다.

단, 한 줄에 5개의 이미지를 보여준다.

In [ ]:
import matplotlib as mpl
def plot_digits(instances, images_per_row=5, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

아래 코드는 손글씨 이미지 총 52,500 훈련 샘플 중 선택된 25개의 이미지를 보여준다.
- 각각의 이미지는 2100개 중에 하나씩 선택되었다.
    - 52,500 = 25 * 2100

In [ ]:
plt.figure(figsize=(7, 4))

# 왼편 원본 손글씨 데이터
plt.subplot(121)
plot_digits(X_train[::2100])
plt.title("Original", fontsize=16)

# 오른편 재구성된 손글씨 데이터
plt.subplot(122)
plot_digits(X_recovered[::2100])
plt.title("Compressed", fontsize=16)

아래 코드는 이어서 설명하는 점진적 PCA 기법의 결과와 비교하기 위해 앞서 설명한 PCA 기법으로 압축된 손글씨 이미지 데이터를 X_reduced_pca로 기억해둔다.

In [ ]:
X_reduced_pca = X_reduced

랜덤 PCA

svd_solver="randomized" 옵션을 사용하면 특이값 분해 알고리즘에 무작위성을 가하여 차원 d만큼의 주성분을 근사적으로 계산한다.

In [ ]:
rnd_pca = PCA(n_components=154, svd_solver="randomized", random_state=42)
X_reduced = rnd_pca.fit_transform(X_train)

각 알고리즘의 시간 복잡도를 비교하면 다음과 같다.
- 기존의 특이값 분해 알고리즘의 시간 복잡도
    - O(m * n^2) + O(n^3)
- 랜덤 특이값 분해 알고리즘의 시간 복잡도
    - O(m * d^2) + O(d^3)

따라서 d가 n보다 많이 작으면 랜덤 PCA가 훨씬 빠르게 작동한다.
이어서 설명하는 점진적 PCA 알고리즘을 소개한 후, 세 알고리즘의 실행속도를 직접 비교한다.

점진적 PCA

주성분 분석 알고리즘은 훈련 세트 전체를 이용한다.
따라서 매우 큰 훈련 세트에 대해서는 메모리와 시간과 관련된 문제가 발생한다.
이 문제를 해결하기 위해 점진적 PCA(IPCA: Incremental PCA)를 사용할 수 있다.
- 미니 배치 학습처럼 지정된 크기의 훈련 데이터(미니 배치)를 차례대로 적재하면서 주성분을 분석한다.
- 온라인 PCA를 구현할 때 유용하다.

아래 코드는 훈련 세트를 쪼개서 미니 배치 단위로 IncrementalPCA 모델을 훈련하는 방법을 보여준다.
- 훈련 세트를 지정된 미니 배치 크기로 쪼갠 후 for 반복문을 실행한다.
- 반복 과정에서 fit()가 아닌 partial_fit()를 반드시 사용해야 한다.

In [ ]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100 # 배치 개수, 미니 배치 크기 = 52,500 / 100 = 525
inc_pca = IncrementalPCA(n_components=154) # 154개의 주성분을 사용하는 모델 지정

for X_batch in np.array_split(X_train, n_batches):
    print(".", end="") # 학습 진도를 보여주는 용도
    inc_pca.partial_fit(X_batch) # partial_fit() 메서드를 사용한다.
X_reduced = inc_pca.transform(X_train) # 154 차원으로 사영한다.

위 방식은 아래와 같이 batch_size 옵션인자와 fit() 메서드를 활용하는 방식과 동일하다.

In [ ]:
batch_size = len(X_train) // n_batches

inc_pca_fit = IncrementalPCA(n_components=154, batch_size=batch_size)
X_reduced_fit = inc_pca_fit.fit_transform(X_train)

실제로 사영 결과가 동일하다

In [ ]:
np.allclose(X_reduced, X_reduced_fit)

재구성 하기
점진적 PCA를 이용해서 사영된 데이터를 재구성한 결과를 비교한다.
- 숫자 인식에는 문제가 없어 보인다.

In [ ]:
X_recovered_inc_pca = inc_pca.inverse_transform(X_reduced)

plt.figure(figsize=(7, 4))

# 왼쪽
plt.subplot(121)
plot_digits(X_train[::2100])

# 오른쪽
plt.subplot(122)
plot_digits(X_recovered_inc_pca[::2100])

plt.tight_layout()

아래 코드는 앞서 설명한 다른 PCA의 결과와 비교하기 위해 점진적 PCA로 사영된 데이터를 기억한다.

In [ ]:
X_reduced_inc_pca = X_reduced

PCA와 IPCA 비교

특이값 분해 알고리즘에 필요한 훈련 세트의 평균값은 동일한 값이 계산된다.

In [ ]:
np.allclose(pca.mean_, inc_pca.mean_)

하지만 사영 결과는 다르다.
- 점진적 PCA 기법의 사영 결과는 상당히 좋기는 하지만 완벽하진 않다.

In [ ]:
np.allclose(X_reduced_pca, X_reduced_inc_pca)

memmap() 함수 사용하기

넘파이의 memmap() 함수를 이용하여 매우 큰 데이터셋을 마치 실제로 메모리에 적재한 것 인양 처리하는 memmap 객체를 생성해서 활용할 수 있다.
처리 과정이 좀 복잡하긴 하지만, 몇 개의 지정된 단계를 거치면 된다.
- memmap은 memory-map의 줄임말이다.
- 실제로는 컴퓨터 저장장치에 저장된 파일을 마치 메모리상의 어레이로 불러온 효과를 보이는 어레이 객체를 가리킨다.

- 1단계: memmap 객체를 생성한 후 훈련 세트와 연결한다.

In [ ]:
filename = "my_mnist.data"
m, n = X_train.shape
X_mm = np.memmap(filename, dtype="float32", mode="write", shape=(m, n)) # memmap 객체 생성
X_mm[:] = X_train # 훈련 세트와 연결한다.

X_mm의 자료형은 memmap이다.

In [ ]:
type(X_mm)

크기는 훈련 세트의 크기와 동일하다.

In [ ]:
X_mm.shape

memmap 객체를 삭제하는 방식으로 더 이상 어레이 조작이 불가능하도록 컴퓨터에 data 확장자를 가진 바이너리 파일로 저장한다.
그러면 현재 파이썬이 실행되는 디렉토리에 my_mnist.data라는 파일이 생성된다.

In [ ]:
del X_mm

- 앞서 동일한 방식으로 저장된 바이너리 파일을 불러온다.
    - 어레이의 모양도 다시 동일하게 지정해야 한다.

In [ ]:
X_mm = np.memmap(
    filename, dtype="float32", mode="readonly", shape=(m, n)
)

배치 크기를 지정한 후, IPCA 모델을 훈련시킨다.
- 여기서는 fit() 메서드를 그대로 사용한다.

In [ ]:
batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

시간 복잡도 비교

앞서 소개된 세 종류의 PCA 알고리즘의 시간 복잡도를 예제를 통해 비교한다.
- 그러면 랜덤 PCA, 일반 PCA, 점진적 PCA 순으로 점점 느려지는 것을 확인할 수 있다.
- 방식은 2차원, 10차원, 154차원으로 사영하는 기준을 따른다.

In [ ]:
import time
for n_components in (2, 10, 154):
    print("n_components = ", n_components)

    # 일반 PCA
    regular_pca = PCA(n_components=n_components, svd_solver="full")

    # 점진적 PCA
    inc_pca = IncrementalPCA(n_components=n_components, batch_size=500)
    
    # 랜덤 PCA
    rnd_pca = PCA(n_components=n_components, random_state=42, svd_solver="randomized")

    for name, pca in (("PCA", regular_pca), ("Inc PCA", inc_pca), ("Rnd PCA", rnd_pca)):
        t1 = time.time()
        pca.fit(X_train)
        t2 = time.time()
        print("{} : {:.1f} seconds".format(name, t2 - t1))

아래 코드는 일반 PCA와 랜덤 PCA를 직접 비교한다.
비교는 차원을 고정시킨 채 훈련 세트의 크기를 증가시키는 방식이며, 일반 PCA 방식이 훨씬 빠름을 보여준다.

In [ ]:
times_rpca = []
times_pca = []
sizes = [1000, 10000, 20000, 30000, 40000, 50000, 70000, 100000, 200000, 500000]
for n_samples in sizes:
    X = np.random.randn(n_samples, 5)
    
    rpca = PCA(n_components=2, svd_solver="randomized", random_state=42)
    t1 = time.time()
    rpca.fit(X)
    t2 = time.time()
    times_rpca.append(t2 - t1)
    
    pca = PCA(n_components=2, svd_solver="full")
    t1 = time.time()
    pca.fit(X)
    t2 = time.time()
    times_pca.append(t2 - t1)

plt.plot(sizes, times_rpca, "b-o", label="RPCA")
plt.plot(sizes, times_pca, "r-s", label="PCA")
plt.xlabel("n_samples")
plt.ylabel("Training time")
plt.legend(loc="upper left")
plt.title("PCA and Randomized PCA time complexity ")

커널 PCA

기본적으로 선형 분류 또는 회귀만을 지원하는 SVM에 커널 기법을 적용하여 비선형 분류와 회귀를 지원하도록 하는 아이디어를 PCA 기법에 적용한다.

커널 기법의 기본 아이디어는 데이터를 고차원으로 보낼 때 갑자기 선형 분류/회귀가 가능해질 수 있다는 것이다.
그런데 동일한 이유로 고차원으로 보내진 데이터셋의 주성분 찾기가 보다 수월해 질 수 있다.
실제로 롤케이크 데이터의 경우 선형 주성분을 찾는 일이 매우 힘들어 보이지만, 커널 기법을 이용하여 고차원으로 보내질 경우, 선형 주성분을 찾는 일이 가능해짐을 아래 가운데에 있는 그래프를 통해 확인할 수 있다.

다양한 커널 기법 적용 예제

아래 코드는 세 종류의 커널을 활용한 결과를 그래프로 보여준다

왼쪽
- 선형 커널, 일반 PCA와 동일한 효과를 낸다.
가운데
- RBF 커널
오른쪽
- 시그모이드 커널

In [ ]:
X, y = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)

from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

lin_pca = KernelPCA(n_components=2, kernel="linear", fit_inverse_transform=True)
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.0433, fit_inverse_transform=True)
sig_pca = KernelPCA(n_components=2, kernel="sigmoid", gamma=0.001, coef0=1, fit_inverse_transform=True)

y = t> 6.9

plt.figure(figsize=(11, 4))
for subplot, pca, title in ((131, lin_pca, "Linear kernel"), (132, rbf_pca, "RBF kernel, $\gamma=0.04$"), (133, sig_pca, "Sigmoid kernel, $\gamma=10^{-3}, r=1$")):
    X_reduced = pca.fit_transform(X)
    if subplot == 132:
        X_reduced_rbf = X_reduced
    
    plt.subplot(subplot)
    #plt.plot(X_reduced[y, 0], X_reduced[y, 1], "gs")
    #plt.plot(X_reduced[~y, 0], X_reduced[~y, 1], "y^")
    plt.title(title, fontsize=14)
    plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=t, cmap=plt.cm.hot)
    plt.xlabel("$z_1$", fontsize=18)
    if subplot == 131:
        plt.ylabel("$z_2$", fontsize=18, rotation=0)
    plt.grid(True)
plt.show()

아래 코드는 앞서 rbf 커널 기법으로 2차원으로 사영된 데이터셋을 다시 3차원으로 복원한 결과를 보여준다.

In [ ]:
plt.figure(figsize=(6, 5))

X_inverse = rbf_pca.inverse_transform(X_reduced_rbf)

ax = plt.subplot(111, projection='3d')
ax.view_init(10, -70)
ax.scatter(X_inverse[:, 0], X_inverse[:, 1], X_inverse[:, 2], c=t, cmap=plt.cm.hot, marker="x")
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_zlabel("")
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])

plt.show()

커널 선택과 하이퍼파라미터 조정

kPCA는 비지도 학습이다. 따라서 어떤 방식, 어떤 커널이 보다 좋은지 평가할 수 있는 기준이 따로 없다.
하지만, 차원축소 이후에 지도학습된 결과를 비교하여 최적의 차원축소 기법과 하이퍼파라미터를 찾을 수는 있다.

예를 들어 아래 코드는 그리드 탐색을 이용하여 최적의 커널 기법과 gamma 옵션의 조합을 찾아낸다.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression(solver="lbfgs"))
])

param_grid=[{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf", "sigmoid"]
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

그리드 탐색 결과 RBF 커널을 gamma=0.043의 조합이 최선임을 알아냈다.

In [ ]:
print(grid_search.best_params_)

찾아낸 최적의 조합을 이용할 때 재구성 오차가 거의 0에 가깝게 나온다.

In [ ]:
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.0433, fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced) # 재구성

from sklearn.metrics import mean_squared_error

mean_squared_error(X, X_preimage) # 재구성 오차가 거의 0이다.

LLE

국소적 선형 임베딩(LLE: Locally Linear Embedding)은 다양한 학습에 의존하는 비선형 차원축소 기법이다.
아래 코드는 롤케이크를 LLE 방식으로 2차원으로 사영한 결과를 보여준다.

In [ ]:
X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=41)

from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_reduced = lle.fit_transform(X)

plt.title("Unrolled swiss roll using LLE", fontsize=14)

plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=t, cmap=plt.cm.hot)

plt.xlabel("$z_1$", fontsize=18)
plt.ylabel("$z_2$", fontsize=18)
plt.axis([-0.065, 0.055, -0.1, 0.12])
plt.grid(True)

plt.show()

기타 차원축소 기법

아래 코드는 기타 차원축소 기법을 사용하는 방식을 보여준다.

사영 기법

선형 판별 분석(LDA)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=2)
X_mnist = mnist["data"]
y_mnist = mnist["target"]
lda.fit(X_mnist, y_mnist)
X_reduced_lda = lda.transform(X_mnist)

다양체 학습 기법

다차원 스케일링(MDS: Multidimensional Scaling)

In [ ]:
from sklearn.manifold import MDS

mds = MDS(n_components=2, random_state=42)
X_reduced_mds = mds.fit_transform(X)

Isomap

In [ ]:
from sklearn.manifold import Isomap

isomap = Isomap(n_components=2)
X_reduced_isomap = isomap.fit_transform(X)

t-SNE(t-distributed Stochastic Neighbor Embedding)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
X_reduced_tsne = tsne.fit_transform(X)

아래 코드는 롤케이크를 MDS, Isomap, t-SNE 방식으로 2차원으로 차원축소한 결과를 보여준다.

In [ ]:
titles=["MDS", "Isomap", "t-SNE"]

plt.figure(figsize=(11, 4))

for subplot, title, X_reduced in zip((131, 132, 133), titles,
                                     (X_reduced_mds, X_reduced_isomap, X_reduced_tsne)):
    plt.subplot(subplot)
    plt.title(title, fontsize=14)
    plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=t, cmap=plt.cm.hot)
    plt.xlabel("$z_1$", fontsize=18)
    if subplot == 131:
        plt.ylabel("$z_2$", fontsize=18, rotation=0)
    plt.grid(True)

plt.show()